In [ ]:
import panel as pn

pn.extension('plotly')

The ``HoloViews`` pane renders HoloViews plots with the selected or explicitly specified backend. It has native support for the regular HoloViews widgets to explore the key dimensions of a ``HoloMap`` or ``DynamicMap``, but is also more flexible since it allows customizing the choice of widget and their position relative to the plot.

#### Parameters:

For layout and styling related parameters see the [layout user guide](../../user_guide/Layout.ipynb).

* **``backend``** (str): Any of the support HoloViews backends ('bokeh', 'matplotlib' or 'plotly')
* **``object``** (object): The HoloViews object being displayed
* **``widget_type``** (str): Whether to generate individual widgets for each dimension or on global scrubber.
* **``widgets``** (dict): A mapping from dimension name to a widget class, instance or dictionary of overrides to modify the default widgets.

##### Display

* **``default_layout``** (pn.layout.Panel, default=Row): Layout to wrap the plot and widgets in

___

When creating any ``HoloViews`` objects the panel function will automatically convert it into a displayable panel, while keeping all interactive features:

In [ ]:
import numpy as np
import holoviews as hv

box = hv.BoxWhisker((np.random.randint(0, 10, 100), np.random.randn(100)), 'Group').sort()

hv_layout = pn.panel(box)
hv_layout

By setting the panes ``object`` the plot can be updated like all other Pane objects:

In [ ]:
hv_layout.object = hv.Violin(box).opts(responsive=True, violin_color='Group', cmap='Category20')

### Widgets

HoloViews natively renders plots with widgets if a HoloMap or DynamicMap declares any key dimensions. Unlike the ``interact`` functionality in panel this efficiently updates just the data inside a plot instead of replacing it entirely. By calling ``pn.panel`` on the DynamicMap will return ``Row`` layout (configurable via the ``default_layout`` option), this is equivalent to calling ``pn.pane.HoloViews(dmap).layout``:

In [ ]:
import pandas as pd
import hvplot.pandas
import holoviews.plotting.bokeh

def sine(frequency=1.0, amplitude=1.0, function='sin'):
    xs = np.arange(200)/200*20.0
    ys = amplitude*getattr(np, function)(frequency*xs)
    return pd.DataFrame(dict(y=ys), index=xs).hvplot()

dmap = hv.DynamicMap(sine, kdims=['frequency', 'amplitude', 'function']).redim.range(
    frequency=(0.1, 10), amplitude=(1, 10)).redim.values(function=['sin', 'cos', 'tan'])

hv_panel = pn.panel(dmap)

hv_panel.pprint()

We can see the widgets generated for each of the dimensions and arrange them any way we like, e.g. by unpacking them into a ``Row``:

In [ ]:
widgets = hv_panel[1]

pn.Column(
    pn.Row(*widgets),
    hv_panel[0])

#### Customizing widgets

As we saw above, the HoloViews pane will automatically try to generate appropriate widgets for the type of data usually defaulting to ``DiscreteSlider`` and ``Select`` widgets. This behavior can be modified by providing a dictionary of ``widgets`` by dimension name, the values of this dictionary can override the default widget in one of three ways:

* Supplying a ``Widget`` instance
* Supplying a compatible ``Widget`` type
* Supplying a dictionary of ``Widget`` parameter overrides
    
A ``Widget`` instances will be used as they are supplied and are expected to provide values matching compatible with the values defined on HoloMap/DynamicMap. Similarly if a ``Widget`` type is supplied it should be discrete if the parameter space defines a discrete set of values, if the defined parameter space is continuous on the other hand it may supply any valid value.


In the example below we override the 'amplitude' dimension with an explicit ``Widget`` instance, the 'function' dimension is overridden with a RadioButtonGroup letting us toggle between the different functions and lastly we override the 'value' parameter on the 'frequency', changing the initial value:

In [ ]:
pn.pane.HoloViews(dmap, widgets={
    'amplitude': pn.widgets.LiteralInput(value=1., type=(float, int)),
    'function': pn.widgets.RadioButtonGroup,
    'frequency': {'value': 5}
}).layout

### Switching backends

The ``HoloViews`` pane will default to the bokeh backend if no backend has been loaded. However

In [ ]:
import holoviews.plotting.mpl
import holoviews.plotting.plotly

hv_pane = pn.pane.HoloViews(dmap, backend='matplotlib')
hv_pane

The ``backend`` like all other parameters can be modified after the fact, to demonstrate we can we can set up a select widget to toggle between backends:

In [ ]:
backend_select = pn.widgets.RadioButtonGroup(name='Backend Selector:', options=['bokeh', 'matplotlib', 'plotly'])
backend_select.link(hv_pane[0], value='backend')
backend_select